In [ ]:
# --- Import Necessary Libraries ---
import praw
import pandas as pd
import time
from datetime import datetime, timedelta
from IPython.display import clear_output

# --- Logging ---
def log(msg):
    print(msg)

def timestamp():
    return datetime.now().strftime('%H:%M:%S')

# --- Reddit API setup ---
reddit = praw.Reddit(
    client_id="0GgjEdXVoxIm0LTkBxIjNA",
    client_secret="NsY4t_AN6o5E4SSki7YkhVee07DCzw",
    user_agent="game-review-bot"
)

# --- Game keywords ---
# --- Game and keywords ---
game_name = "monster hunter wilds"
subreddit = "games"  # only one subreddit

keywords = [
    "monster hunter wilds"]

# --- Date range: daily from release onward ---
start_date = datetime(2025, 2, 28)
end_date = datetime(2025, 3, 31)
step = timedelta(days=1)

# --- Storage ---
posts_data = []
comments_data = []
deduped_post_ids = set()

# --- Scraper loop ---
for game, keywords in games.items():
    for keyword in keywords:
        for subreddit in subreddits:
            current_start = start_date

            while current_start < end_date:
                current_end = current_start + step
                start_ts = int(current_start.timestamp())
                end_ts = int(current_end.timestamp())

                log(f"[{timestamp()}] 🔍 Scraping '{keyword}' for {game} in r/{subreddit} from {current_start.date()} to {current_end.date()}")

                try:
                    for post in reddit.subreddit(subreddit).search(keyword, sort="new", time_filter="all", limit=100):
                        time.sleep(1.5)

                        if not (start_ts <= post.created_utc < end_ts):
                            continue
                        if post.id in deduped_post_ids:
                            continue

                        full_text = f"{post.title} {post.selftext}".lower()
                        is_strong_match = "monster hunter wilds" in full_text
                        title_body_keywords = [kw for kw in keywords if kw in full_text]

                        matched_in_comments = False
                        comment_matches = []

                        try:
                            post.comments.replace_more(limit=None)
                            all_comments = post.comments.list()
                            log(f"    🗨️ {len(all_comments)} comments in post {post.id}")

                            for i, comment in enumerate(all_comments):
                                if any(kw in comment.body.lower() for kw in keywords):
                                    matched_in_comments = True
                                    comment_matches.append({
                                        "game": game,
                                        "keyword": keyword,
                                        "subreddit": subreddit,
                                        "post_id": post.id,
                                        "comment_id": comment.id,
                                        "comment_body": comment.body,
                                        "author": str(comment.author),
                                        "score": comment.score,
                                        "created_utc": comment.created_utc
                                    })
                                if i % 20 == 0 and i != 0:
                                    log(f"      ⏳ Scanned {i+1} comments...")

                        except Exception as e:
                            log(f"⚠️ Error fetching comments for post {post.id}: {e}")

                        # Apply refined match filter
                        if is_strong_match or len(title_body_keywords) >= 2 or matched_in_comments:
                            deduped_post_ids.add(post.id)
                            posts_data.append({
                                "game": game,
                                "keyword": keyword,
                                "subreddit": subreddit,
                                "post_id": post.id,
                                "title": post.title,
                                "selftext": post.selftext,
                                "score": post.score,
                                "num_comments": post.num_comments,
                                "created_utc": post.created_utc,
                                "author": str(post.author),
                                "matched_keywords_title_body": title_body_keywords
                            })

                            comments_data.extend(comment_matches)

                except Exception as e:
                    log(f"⛔ Error scraping r/{subreddit} for {keyword} on {current_start.date()} → {e}")

                current_start = current_end

In [ ]:
posts_df = pd.DataFrame(posts_data)
comments_df = pd.DataFrame(comments_data)

posts_df.to_csv("reddit_MHW_Posts331.csv", index=False)
comments_df.to_csv("reddit_MHW_Comments.csv", index=False)

log(f"\n✅ Done! Saved {len(posts_df)} posts and {len(comments_df)} comments.")